# Quantum Circuit Partitioning with FM Walkthrough

This is a brief intro to the contents of the repository, showing a basic workflow for partitioning a quantum circuit across multiple QPUs.

The partitioning has a number of phases, some of which can be modified or removed in a modular fashion.

We are given a circuit and quantum network information by the user and we want to return a partitioned circuit which is compatible with the network, and minimal in terms of the required entanglement.

Typically we have the phases:

1. Transpilation

The circuit is transpiled into a particular gate set. We recommend the $[ U(\theta, \phi, \lambda), CP(\theta)]$ gateset, since , which is used by default.

2. Graph conversion

The circuit is converted into a "temporal graph" representing qubit interactions over time.

3. Gate grouping (optional)

Edges on the graph are merged into hyper-edges based on their compatibility for gate teleportation. 

3. Coarsening (optional) 

The graph is coarsened into a graph which is smaller and easier to optimise.

4. Partitioning and refinment 

The graph is partitioned using the FM algorithm. If the graph has been coarsened, the partitioning is refined at each level until the original graph is recovered.

6. Circuit extraction

A quantum circuit is extracted, containing all the new information for qubit and gate teleportation.


## Transpilation

We begin by defining a circuit and using the built in qiskit transpiler to put it into the (u,cp) gate-set.

In [9]:
from qiskit.circuit.library import QFT, QuantumVolume
from disqco.circuits.cp_fraction import cp_fraction, cz_fraction
from qiskit import transpile, QuantumCircuit
from disqco.graphs.quantum_network import QuantumNetwork
import numpy as np


num_qubits = 112
seed = np.random.randint(0,10000)
print(f'Seed: {seed}')
circuit = cp_fraction(num_qubits, depth=num_qubits, fraction=0.5, seed=seed)


# circuit = QuantumCircuit(num_qubits)

# circuit.cp(np.pi,0,1)

# circuit = QFT(num_qubits, do_swaps=False)
# circuit = QuantumVolume(num_qubits, depth=3)

num_partitions = 2 # Define the number of partitions or QPUs you will distribute over
depth = circuit.depth()

qpu_size = num_qubits // num_partitions + 1 # Define the number of qubits per QPU. For simplicity, we divide qubits evenly and add one additional space to each for teleportation (an extra space is needed for the teleportation qubit so the algorithmr requires it!)
qpu_sizes = [qpu_size] * num_partitions # Store the number of qubits per QPU in a list

# Create a quantum network with the specified number of qubits and partitions

# If we do not specificy the connectivity, we have all-to-all by default.

quantum_network = QuantumNetwork(qpu_sizes)


# circuit.draw(output='mpl')
 

Seed: 2214


In [10]:
basis_gates = ['u', 'cp']

# # Transpile the circuit to the basis gates
circuit = transpile(circuit, basis_gates=basis_gates)

print(f'Number of qubits in circuit {circuit.num_qubits}')
print(f'Circuit depth: {circuit.depth()}')

from disqco.graphs.QC_hypergraph import QuantumCircuitHyperGraph
from disqco.parti.FM.FM_methods import set_initial_partition_assignment

depth = circuit.depth() # Get the depth of the circuit

graph = QuantumCircuitHyperGraph(circuit, group_gates=True, anti_diag=True, map_circuit=True)

assignment = set_initial_partition_assignment(graph, quantum_network)

# # circuit.draw(output='mpl')


Number of qubits in circuit 112
Circuit depth: 112


# Graph conversion

We now convert the circuit to the GCP Hypergraph using a number of predefined functions which can be found in the src code.

There is an optional parameter in the 'map_circuit_to_hypergraph' function called 'group_gates'. If this is set to false, then we get a regular graph with state edges connecting qubit nodes at successive time steps, and state edges for each two-qubit gate. If group_gates is set to true, then a pass is applied which merges some of these gate edges into hyper-edges. This makes the graph structure a bit more complex, so avoid it for now. We can tackle that later.

We will also set the 'assignment' which is our problem variable. It's a matrix of dimension $depth \times num\_qubits$. Assignment[t][q] indicates the partition qubit $q$ is assigned to at time $t$. This corresponds to node $(q,t)$. The initial assignment just maps qubits in a greedy fashion, squeezing qubits onto the first QPU until its full then moving to the next. The assignment at $t=0$ is then used for the full depth.

In [11]:
from disqco.drawing.tikz_drawing import draw_graph_tikz

%load_ext jupyter_tikz

# graph.draw(network=quantum_network, assignment=assignment)

The jupyter_tikz extension is already loaded. To reload it, use:
  %reload_ext jupyter_tikz


## Partitioning

First we consider standard partitioning with the FM algorithm (i.e. no Coarsening).

In [12]:
from disqco.parti.FM.fiduccia import FiducciaMattheyses

# node_counts = len(graph.nodes)
# move_limit = num_qubits

partitioner = FiducciaMattheyses(circuit,
                                 network=quantum_network,
                                 initial_assignment=assignment,
                                 hypergraph=graph)

results = partitioner.multilevel_partition(passes_per_level=10,stochastic=True)

final_cost, final_assignment = results['best_cost'], results['best_assignment']

print(f'Final cost: {final_cost}')

# Check final cost

from disqco.parti.FM.FM_methods import calculate_full_cost

calculated_cost = calculate_full_cost(graph, final_assignment, num_partitions)
print(f'Calculated cost: {calculated_cost}')



Final cost: 654
Calculated cost: 654


If you want you can use the drawing tool to see what the new solution looks like, where the nodes are dragged around to match the assignment. The assignment variable will have to be replaced with final_assignment now! If it's working properly you should see two (or however many partitions you have) sub-graphs which are connected only by the non-local/cutting edges. You can count them to make sure it corresponds to the final cost (although this gets messy for big graphs and when hyper-edges are introduced).

In [13]:
graph.draw(network=quantum_network, assignment=final_assignment)

KeyboardInterrupt: 

In most cases, we can get a better result and a quicker solution using some coarsening. In this example, we use a recursive coarsening method, which is the most effective from the paper. The coarsening and partitioning is all taken care of by the "MLFM_recursive" function. See the coarsening notebook see how it works and to explore other methods.

In [ ]:
from disqco.graphs.coarsening.coarsener import HypergraphCoarsener
import numpy as np
import time
from disqco.parti.FM.fiduccia import FiducciaMattheyses

recursive_coarsener = HypergraphCoarsener().coarsen_recursive_batches

start = time.time()
results = partitioner.multilevel_partition(coarsener=recursive_coarsener)
end = time.time()

print(f'Time taken for MLFM: {end - start} seconds')


final_cost_MLFMR = results['best_cost']
final_assignment_MLFMR = results['best_assignment']
print(f'Final cost: {final_cost_MLFMR}')

if final_cost_MLFMR < final_cost:
    print("Final cost is smaller than FM cost, using MLFM assignment")
    best_assignment = final_assignment_MLFMR


Time taken for MLFM: 2.033663034439087 seconds
Final cost: 164


In [ ]:
graph.draw(network=quantum_network, assignment=best_assignment)

NameError: name 'best_assignment' is not defined

# Circuit extraction



In [ ]:
from disqco.circuit_extraction.circuit_extractor import PartitionedCircuitExtractor

extractor = PartitionedCircuitExtractor(graph=graph, 
                                        network=quantum_network,
                                        partition_assignment=best_assignment)

final_circuit = extractor.extract_partitioned_circuit()

In [ ]:
final_circuit = transpile(final_circuit, basis_gates=basis_gates + ['EPR'])

print("Number of EPR pairs in partitioned circuit: ", final_circuit.count_ops()['EPR'])

Number of EPR pairs in partitioned circuit:  857


In [ ]:
from disqco.circuit_extraction.verification import run_sampler, plot

measured_circuit = circuit.copy()
measured_circuit.measure_all()

data_circ = run_sampler(measured_circuit, shots=10000)

plot(data_circ)

Too many qubits
No data to plot


In [ ]:
data_circ_part = run_sampler(final_circuit, shots=10000)

plot(data_circ_part)

Too many qubits
No data to plot


In [ ]:
final_circuit.draw(output='mpl', style = 'bw', fold=100) # Draw the circuit

See circuit extraction notebook for more!